In [1]:
!pip install boto3
!pip install pandas
!pip install numpy
!pip install sqlalchemy
!pip install psycopg2
!pip install pymysql

In [2]:
import boto3
import pandas as pd
import numpy as np
import pymysql
import psycopg2
import configparser
from sqlalchemy import create_engine


In [33]:
config = configparser.ConfigParser()
try:
    config.read('config.conf')
    if not config.sections():
        print("No se encontraron secciones en el archivo de configuración.")
    else:
        print("Archivo de configuración cargado correctamente.")
except Exception as e:
    print("Ocurrió un error al leer el archivo de configuración:", e)
config

Archivo de configuración cargado correctamente.


In [34]:
aws_rds_conn = boto3.client('rds',
                             aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                             aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                             region_name=config.get('IAM', 'REGION'))



In [35]:
aws_ec2_conn = boto3.client('ec2',
                             aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                             aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                             region_name=config.get('IAM', 'REGION'))


In [10]:
try:
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'),
        DBName=config.get('TRANSACC', 'DB_NAME'),
        DBInstanceClass='db.t3.micro',
        Engine='mysql',
        MasterUsername=config.get('TRANSACC', 'DB_USER'),
        MasterUserPassword=config.get('TRANSACC', 'DB_PASSWORD'),
        Port=int(config.get('TRANSACC', 'DB_PORT')),
        PubliclyAccessible=True,
        VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
        AllocatedStorage=20
    )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La instancia de base de datos MySQL ya existe.")
except Exception as ex:
    print("Error al crear la base de datos MySQL:", ex)


{'DBInstance': {'DBInstanceIdentifier': 'transacc-market', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'creating', 'MasterUsername': 'admin', 'DBName': 'market', 'AllocatedStorage': 20, 'PreferredBackupWindow': '10:10-10:40', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-095a5320e579c264c', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-0d0097d620899bdc0', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-00af2d95e956f0117', 'SubnetAvailabilityZone': {'Name': 'us-east-1f'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-082fe405ea6cb9552', 'SubnetAvailabilityZone': {'Name': 'us-east-1b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0589815

In [20]:
# Crear la base de datos PostgreSQL para DWH
try:
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=config.get('DWH', 'DB_INSTANCE_ID'),
        DBName=config.get('DWH', 'DB_NAME'),
        DBInstanceClass='db.t3.micro',
        Engine='postgres',
        MasterUsername=config.get('DWH', 'DB_USER'),
        MasterUserPassword=config.get('DWH', 'DB_PASSWORD'),
        Port=int(config.get('DWH', 'DB_PORT')),
        PubliclyAccessible=True,
        VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
        AllocatedStorage=20
    )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La instancia de base de datos PostgreSQL ya existe.")
except Exception as ex:
    print("Error al crear la base de datos PostgreSQL:", ex)


{'DBInstance': {'DBInstanceIdentifier': 'dwh-market', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'postgresql', 'DBName': 'dwh_market', 'AllocatedStorage': 20, 'PreferredBackupWindow': '09:30-10:00', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-095a5320e579c264c', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-0d0097d620899bdc0', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-00af2d95e956f0117', 'SubnetAvailabilityZone': {'Name': 'us-east-1f'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-082fe405ea6cb9552', 'SubnetAvailabilityZone': {'Name': 'us-east-1b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subne

In [28]:
def generate_key_pair(key_name):    
    ec2_client = boto3.client('ec2',
                              aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                              aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                              region_name=config.get('IAM', 'REGION'))
    
    try:
        key_pair = ec2_client.create_key_pair(KeyName=key_name)
        private_key = key_pair['KeyMaterial']
        
        # Guardar la clave privada en un archivo
        with open(f'{key_name}.pem', 'w') as file:
            file.write(private_key)
        
        print(f'Key pair "{key_name}" creado exitosamente.')
        print(f'Clave privada guardada en: {key_name}.pem')
    
    except ec2_client.exceptions.ClientError as e:
        if 'InvalidKeyPair.Duplicate' in str(e):
            print(f'El key pair "{key_name}" ya existe.')
        else:
            print('Error al crear el key pair:', e)


# Nombre del key pair a generar
key_name = 'key-pair-project'

# Generar el key pair
generate_key_pair(key_name)

Key pair "key-pair-project" creado exitosamente.
Clave privada guardada en: key-pair-project.pem


In [36]:
try:
    response = aws_ec2_conn.run_instances(
        ImageId=config.get('EC2', 'IMAGE_ID'),
        InstanceType='t2.micro',
        KeyName=config.get('EC2', 'KEY_PAIR'),
        MinCount=1,
        MaxCount=1,
        SecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')]
    )
    instance_id = response['Instances'][0]['InstanceId']
    print("Instancia EC2 creada con ID:", instance_id)
except Exception as ex:
    print("Error al crear la instancia EC2:", ex)


Instancia EC2 creada con ID: i-00d9271280e541f4a


In [37]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'))
    mysql_hostname = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print("Hostname de la base de datos MySQL:", mysql_hostname)
except Exception as ex:
    print("Error al obtener el hostname de la base de datos MySQL:", ex)


Hostname de la base de datos MySQL: transacc-market.cvys8o2w4r4x.us-east-1.rds.amazonaws.com


In [38]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('DWH', 'DB_INSTANCE_ID'))
    postgresql_hostname = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print("Hostname de la base de datos PostgreSQL:", postgresql_hostname)
except Exception as ex:
    print("Error al obtener el hostname de la base de datos PostgreSQL:", ex)
    

Hostname de la base de datos PostgreSQL: dwh-market.cvys8o2w4r4x.us-east-1.rds.amazonaws.com


In [39]:
# Eliminar la base de datos MySQL de transacciones
try:
    response = aws_rds_conn.delete_db_instance(
        DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'),
        SkipFinalSnapshot=True
    )
    print(response)
except Exception as ex:
    print("Error al eliminar la base de datos MySQL:", ex)


{'DBInstance': {'DBInstanceIdentifier': 'transacc-market', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'deleting', 'MasterUsername': 'admin', 'DBName': 'market', 'Endpoint': {'Address': 'transacc-market.cvys8o2w4r4x.us-east-1.rds.amazonaws.com', 'Port': 3306, 'HostedZoneId': 'Z2R2ITUGPM61AM'}, 'AllocatedStorage': 20, 'InstanceCreateTime': datetime.datetime(2024, 4, 11, 6, 24, 35, 399000, tzinfo=tzlocal()), 'PreferredBackupWindow': '10:10-10:40', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-095a5320e579c264c', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'AvailabilityZone': 'us-east-1f', 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-0d0097d620899bdc0', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-00af2d95e956f0117', 'SubnetAvailabilityZone

In [40]:
# Eliminar la base de datos PostgreSQL para DWH
try:
    response = aws_rds_conn.delete_db_instance(
        DBInstanceIdentifier=config.get('DWH', 'DB_INSTANCE_ID'),
        SkipFinalSnapshot=True
    )
    print(response)
except Exception as ex:
    print("Error al eliminar la base de datos PostgreSQL:", ex)



{'DBInstance': {'DBInstanceIdentifier': 'dwh-market', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'deleting', 'MasterUsername': 'postgresql', 'DBName': 'dwh_market', 'Endpoint': {'Address': 'dwh-market.cvys8o2w4r4x.us-east-1.rds.amazonaws.com', 'Port': 5432, 'HostedZoneId': 'Z2R2ITUGPM61AM'}, 'AllocatedStorage': 20, 'InstanceCreateTime': datetime.datetime(2024, 4, 11, 6, 25, 50, 674000, tzinfo=tzlocal()), 'PreferredBackupWindow': '09:30-10:00', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-095a5320e579c264c', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16', 'ParameterApplyStatus': 'in-sync'}], 'AvailabilityZone': 'us-east-1c', 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-0d0097d620899bdc0', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-00af2d95e956f0117', 'SubnetAvailability

In [41]:
# Eliminar la instancia EC2
try:
    response = aws_ec2_conn.terminate_instances(
        InstanceIds=[instance_id]
    )
    print(response)
except Exception as ex:
    print("Error al eliminar la instancia EC2:", ex)

{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-00d9271280e541f4a', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': 'c24ad799-3524-4287-98ad-be3289bd391c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c24ad799-3524-4287-98ad-be3289bd391c', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '594', 'date': 'Thu, 11 Apr 2024 06:37:50 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


In [42]:
try:
    response = aws_ec2_conn.delete_key_pair(
        KeyName=config.get('EC2', 'KEY_PAIR')
    )
    print(response)
except Exception as ex:
    print("Error al eliminar el par de claves:", ex)

{'Return': True, 'KeyPairId': 'key-06954ef689b2fb5bf', 'ResponseMetadata': {'RequestId': 'acace98c-5cde-4368-a68d-443a2c5c80eb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'acace98c-5cde-4368-a68d-443a2c5c80eb', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '276', 'date': 'Thu, 11 Apr 2024 06:38:08 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
